In [1]:
%matplotlib inline

from ROOT import TFile,vector
import ROOT
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from numpy import mean
from math import sqrt,acos,cos,sin,pi,exp,log,isnan,atan2
from numpy import asarray
from root_pandas import read_root
from matplotlib import gridspec
from scipy import stats
import seaborn as sns

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import asarray

Welcome to JupyROOT 6.18/04


In [2]:
# This file loads up our samples, applies a cosmic bdt weight and removes all vertices assigned to the same event
# except for the one with the highest cosmic likelihood score
# Basically, we only want one vertex per event, so this does that.

## Load  Things Up

In [5]:
datatag = 'Mar4-WC'
mctag = 'Mar30-WC'

df_run1_presel = pd.read_parquet('../data/pickles/data_goodruns_precuts_%s.parquet'%datatag)
df_run3_presel = pd.read_parquet('../data/pickles/datarun3_goodruns_precuts_%s.parquet'%datatag)
df_numu_presel = pd.read_parquet('../data/pickles/numu_goodruns_precuts_%s.parquet'%mctag)
df_nue_presel = pd.read_parquet('../data/pickles/nue_goodruns_precuts_%s.parquet'%mctag)
df_ext_presel = pd.read_parquet('../data/pickles/ext_goodruns_precuts_%s.parquet'%datatag)
df_dirt_presel = pd.read_parquet('../data/pickles/dirt_goodruns_precuts_%s.parquet'%mctag)

# Kill Duplicates

In [6]:
# Let's kill some dupes
with open('/home/dcianci/Physics/1e1p/1mu1pSelection/cosmicbdt_1mu1p_mar31_ccqe.pickle','rb') as handle: cosmicBDT = pickle.load(handle)          # Load BDT weights for 1mu1p background differentiation

vars_cos_allofem = ['Eta','ChargeNearTrunk','PT_1m1p','PhiT_1m1p','AlphaT_1m1p','Sph_1m1p','Q0_1m1p','Q2_1m1p','Q3_1m1p','Lepton_ThetaReco','Lepton_PhiReco','Proton_ThetaReco','Proton_PhiReco','PTRat_1m1p','Lepton_TrackLength','Thetas','Phis','Proton_TrackLength','OpenAng','PzEnu_1m1p']

myvars = vars_cos_allofem
scorename = 'CosBkgScore'

df_run1_presel.insert(0,scorename,cosmicBDT.predict(df_run1_presel[myvars].values.tolist(),output_margin=True))
df_run3_presel.insert(0,scorename,cosmicBDT.predict(df_run3_presel[myvars].values.tolist(),output_margin=True))
df_numu_presel.insert(0,scorename,cosmicBDT.predict(df_numu_presel[myvars].values.tolist(),output_margin=True))
df_nue_presel.insert(0,scorename,cosmicBDT.predict(df_nue_presel[myvars].values.tolist(),output_margin=True))
df_ext_presel.insert(0,scorename,cosmicBDT.predict(df_ext_presel[myvars].values.tolist(),output_margin=True))
df_dirt_presel.insert(0,scorename,cosmicBDT.predict(df_dirt_presel[myvars].values.tolist(),output_margin=True))

In [7]:
df_run1_nodupes = df_run1_presel.sort_values('CosBkgScore',ascending=False).drop_duplicates(['run','subrun','event']).sort_index()
df_run3_nodupes = df_run3_presel.sort_values('CosBkgScore',ascending=False).drop_duplicates(['run','subrun','event']).sort_index()
df_numu_nodupes = df_numu_presel.sort_values('CosBkgScore',ascending=False).drop_duplicates(['run','subrun','event']).sort_index()
df_nue_nodupes = df_nue_presel.sort_values('CosBkgScore',ascending=False).drop_duplicates(['run','subrun','event']).sort_index()
df_ext_nodupes = df_ext_presel.sort_values('CosBkgScore',ascending=False).drop_duplicates(['run','subrun','event']).sort_index()
df_dirt_nodupes = df_dirt_presel.sort_values('CosBkgScore',ascending=False).drop_duplicates(['run','subrun','event']).sort_index()

In [8]:
tag = 'Mar31-WC'
df_dirt_nodupes.to_parquet('../data/pickles/dirt_nodupes_%s.parquet'%tag)
df_run1_nodupes.to_parquet('../data/pickles/datarun1_nodupes_%s.parquet'%tag)
df_run3_nodupes.to_parquet('../data/pickles/datarun3_nodupes_%s.parquet'%tag)
df_numu_nodupes.to_parquet('../data/pickles/numu_nodupes_%s.parquet'%tag)
df_nue_nodupes.to_parquet('../data/pickles/nue_nodupes_%s.parquet'%tag)
df_ext_nodupes.to_parquet('../data/pickles/ext_nodupes_%s.parquet'%tag)